In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
base_url = "https://aarhusbolig.dk"

In [3]:
def get_next_page(html):
    try:
        search_options_bar = html.find("div", class_="search-options-bar l-document")
        nxt = search_options_bar.find("span", "icon-arrow-right prev-next-icon")
        return int(nxt.parent["href"].split("&")[-1].split("=")[1])
    except:
        return None

In [4]:
def get_urls(html):
    container = html.find("div",  {"class": ["housecard-slider w-housecards desktop-only-large"]})
    links = container.find_all("a", class_="housecard-img")
    return [f"{base_url}{a['href']}" for a in links]

In [5]:
def get_page(page):
    url = f"{base_url}/Soeg-bolig?Fritekst=&omr-data-carrier=&BoligOmraader=&org-data-carrier=&Selskaber=&bolig-data-carrier=&BoligTyper=&FloorWish.Min=&FloorWish.Max=&min-data-carrier=&HuslejeMin=&max-data-carrier=&HuslejeMax=&antv-data-carrier=&AntalVaerelser=&antvmax-data-carrier=&AntalVaerelserMax=&div-data-carrier=&BoligArealMin=0&BoligArealMax=200&orderby=&sort=&page={page}"
    result = requests.get(url)
    return BeautifulSoup(result.content, "html.parser")

In [6]:
all_urls = []
next_page = 1
while next_page != None:
    content = get_page(next_page)
    links = get_urls(content)
    print(f"Page {next_page}")
    all_urls += links
    next_page = get_next_page(content)
print(f"DONE! {len(all_urls)} listings")

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42
Page 43
Page 44
Page 45
Page 46
Page 47
Page 48
Page 49
Page 50
Page 51
Page 52
Page 53
Page 54
Page 55
Page 56
Page 57
Page 58
Page 59
Page 60
Page 61
Page 62
Page 63
Page 64
Page 65
Page 66
Page 67
Page 68
Page 69
Page 70
Page 71
Page 72
Page 73
Page 74
Page 75
Page 76
Page 77
Page 78
Page 79
Page 80
Page 81
Page 82
Page 83
Page 84
Page 85
Page 86
Page 87
Page 88
Page 89
Page 90
Page 91
Page 92
Page 93
Page 94
Page 95
Page 96
Page 97
Page 98
Page 99
Page 100
Page 101
Page 102
Page 103
Page 104
Page 105
Page 106
Page 107
Page 108
Page 109
Page 110
Page 111
Page 112
Page 113
Page 114
Page 115
Page 116
Page 117
Page 118
Page 119
Page 120
Page 121
Page 122
Page 123
P

In [7]:
def get_listing(url):
    result = requests.get(url)
    return BeautifulSoup(result.content, "html.parser")

In [9]:
def to_int(text):
    return int(text.split(" ")[0].split(",")[0].replace(".", ""))

In [10]:
def get_data(listing_url):
    listing_html = get_listing(listing_url)
    details = listing_html.find("div", class_="bolig-details")
    data = {}
    data["url"] = listing_url
    data["address"] = details.find("h1").text
    base_data = details.find("table", class_="afdeling-bolig-data")

    try:
        span = listing_html.find("span", text="Boligorganisation").parent.find("a")["href"]
        data["company"] = re.search(r'www\.(.*)\.', span).groups()[0]
    except:
        print("Failed company", listing_url)
        pass
        
    tds = base_data.find_all("td")
    data["type"] = tds[0].text
    data["rooms"] = to_int(tds[1].text)
    data["listing"] = to_int(tds[2].text)
    data["size_m2"] = to_int(tds[3].text)
    
    for cell in tds[4].find_all("div"):
        spans = cell.find_all("span")
        data[spans[0].text] = to_int(spans[1].text)
    data["price"] = to_int(base_data.find("span", class_="details-last details-sum").text)
    
    try:
        data["lat"], data["long"] = listing_html \
            .find("table", class_="afdeling-bolig-data") \
            .find("span", text="Se på kort") \
            .parent['href'].split("|")[-1].split(",")
        
        # Swapping on purpose to create an unclean dataset
        data["long"], data["lat"] = data["lat"], data["long"]
    except:
        print("Failed location", listing_url)
        pass
    
    data["text"] = listing_html.find("ul", class_="afdeling-details").text.replace("\r", "").replace("\n", "")
    return data

In [11]:
listings = []
total = len(all_urls)
current = 1
for listing_url in all_urls:
    print(f"{current}/{total}")
    listing = get_data(listing_url)
    listings.append(listing)
    current += 1

1/2113
2/2113
3/2113
4/2113
5/2113
6/2113
7/2113
8/2113
9/2113
10/2113
11/2113
12/2113
13/2113
14/2113
15/2113
16/2113
17/2113
18/2113
19/2113
20/2113
21/2113
22/2113
23/2113
24/2113
25/2113
26/2113
27/2113
28/2113
29/2113
30/2113
31/2113
32/2113
33/2113
34/2113
35/2113
36/2113
37/2113
38/2113
39/2113
40/2113
41/2113
42/2113
43/2113
44/2113
45/2113
46/2113
47/2113
48/2113
49/2113
50/2113
51/2113
52/2113
53/2113
54/2113
55/2113
56/2113
57/2113
58/2113
59/2113
60/2113
61/2113
62/2113
63/2113
64/2113
65/2113
66/2113
67/2113
68/2113
69/2113
70/2113
71/2113
72/2113
73/2113
74/2113
75/2113
76/2113
77/2113
78/2113
79/2113
80/2113
81/2113
82/2113
83/2113
84/2113
85/2113
86/2113
87/2113
88/2113
89/2113
90/2113
91/2113
92/2113
93/2113
94/2113
95/2113
96/2113
97/2113
98/2113
99/2113
100/2113
101/2113
102/2113
103/2113
104/2113
105/2113
106/2113
107/2113
108/2113
109/2113
110/2113
111/2113
112/2113
113/2113
114/2113
115/2113
116/2113
117/2113
118/2113
119/2113
120/2113
121/2113
122/2113
123/2113
1

In [12]:
import json
jsonString = json.dumps(listings)
jsonFile = open("data.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

# 2. Loading the data

In [13]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)

In [14]:
df = pd.read_json('data.json')

In [15]:
df2 = pd.DataFrame()
df2["listingid"] = df["url"].str.extract(r'bolignr=([0-9]*)')
df2["companyid"] = df["url"].str.extract(r'selskab=([0-9]*)')
df2["company"] = df["company"]
df2["price"] = df["price"]
df2["url"] = df["url"]

df2["long"] = df["long"]
df2["lat"] = df["lat"]

df2["address"] = df["address"]
df2["type"] = df["type"]
df2["rooms"] = df["rooms"]
df2["size_m2"] = df["size_m2"]
df2["listings"] = df["listing"]
df2["text"] = df["text"]

In [16]:
others = [
    'Råderet',
    'Hjemfald',
    'Merleje køkken/bad', 
    'Indv.vedlh',
    'Merinvestering solceller',
    'Merleje flisebelægning',
    'Merleje overdækning','Diverse',
    'Tillæg stor affaldsspand',
    'Etablering Bolignet Århus',
    'Vinduespolering',
    'Etablering af p-pladser',
    'Garage',
    'Afskr. eksisterende køkken',
    'Afskr. eksisterende bad',
    'Afskr. eksisterende skab bad', 
    'Modernisering køkken/bad',
    'Øvrig modernisering', 
    'Råderetsarbejde', 
    'Ydelse indv.mod.', 
    'Vedl. Andel', 
    'Skorstensfejning', 
    'Individuel råderet', 
    'Badmodernisering', 
    'Køkkenmodernisering'
]

colmap = {
    "deposit": ["Indskud:"],
    "rent": ["Boligleje", "Leje", "Nettohusleje", 'Husleje', 'Nettoleje', 'Boligleje, hens.vedl.konto'],
    "electricity": ["Aconto el", 'A conto el', 'ac el', 'A/c el', 'El', 'A/C El', 'A-C el'],
    "heat": ["Aconto varme", 'A conto varme', 'Varme', 'A-C varme', 'A/C Varme', 'ac varme'],
    "water": ["Aconto vand", "A/c vand", 'A conto vand', 'Vand', 'ac vand', 'A/c varme', 'A/C Koldt vand', 'A/C Varmt vand', 'A-C vand', 'Aconto vand/Vandafled.'],
    "tv_internet": ['Antenneafgift', 'Internet, telenet, bredbånd eller bolignet Århus', 'Kabel Tv', 'Antenne/internet', 'Antennebidrag', 'Antenne'],
    "b-ordning": ['B-ordning', 'B-ordn'],
    "other": others
}

In [17]:
for key in colmap:
    df2[key] = df[colmap[key]].sum(axis=1)
df2.replace(0, np.nan, inplace=True)

In [18]:
df2.to_csv("aarhusbolig.csv", index=False)